In [6]:
import boto3
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

In [31]:
client = boto3.client('opensearchserverless')
service = 'aoss'
region = 'us-east-1'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, service, session_token=credentials.token)

In [39]:
client = OpenSearch(
        hosts=[{'host': "-", 'port': 443}],
        http_auth=awsauth,
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        timeout=300
    )

In [54]:
client.indices.get_alias()

{'.opensearch_dashboards_1': {'aliases': {'.opensearch_dashboards': {}}}}

In [55]:
client.indices.exists(index='hubsync-ai-assistant')

False

In [49]:
client.indices.delete(index='hubsync-ai-assistant')

{'acknowledged': True}

In [53]:
client.indices.get(index='hubsync-ai-assistant')

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [hubsync-ai-assistant]')

In [17]:
BODY_CREATE_INDEX_DEFAULT = {
        "aliases": {},
        "mappings": {
            "properties": {
                "metadata": {
                    "properties": {
                        "session_id": {
                            "type": "text",
                            "fields": {
                                "keyword": {"type": "keyword", "ignore_above": 256}
                            },
                        },
                        "source": {
                            "type": "text",
                            "fields": {
                                "keyword": {"type": "keyword", "ignore_above": 256}
                            },
                        },
                    }
                },
                "text": {
                    "type": "text",
                    "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
                },
                "vector_field": {
                    "type": "knn_vector",
                    "dimension": 1536,
                    "method": {
                        "engine": "nmslib",
                        "space_type": "l2",
                        "name": "hnsw",
                        "parameters": {"ef_construction": 512, "m": 16},
                    },
                },
            }
        },
        "settings": {
            "index": {
                "number_of_shards": "1",
                "knn.algo_param": {"ef_search": "512"},
                "knn": "true",
                "number_of_replicas": "1",
            }
        },
    }

In [18]:
client.indices.create(
    index='hubsync-v2',
    body=BODY_CREATE_INDEX_DEFAULT
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'hubsync-v2'}

In [58]:
client.indices.get(index='hubsync-ai-assistant')

{'hubsync-ai-assistant': {'aliases': {},
  'mappings': {'properties': {'id': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'metadata': {'properties': {'session_id': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'source': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'text': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'vector_field': {'type': 'knn_vector',
     'dimension': 1536,
     'method': {'engine': 'nmslib',
      'space_type': 'l2',
      'name': 'hnsw',
      'parameters': {'ef_construction': 512, 'm': 16}}}}},
  'settings': {'index': {'number_of_shards': '2',
    'knn.algo_param': {'ef_search': '512'},
    'provided_name': 'hubsync-ai-assistant',
    'knn': 'true',
    'creation_date': '1697142184602',
    'number_of_replicas': '0',
    'uuid': 'dOOOJYsBxLQ7ROhYmTQL',
    'version': {'c

In [60]:
client.search(
    index="hubsync-ai-assistant",
    body={
        "query": {
            "bool": {
                "must": {
                    "match": {
                        "metadata.session_id":  "123456"
                        }
                    }
                }
            }
        }
)

{'took': 235,
 'timed_out': False,
 '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': 0.18232156,
  'hits': [{'_index': 'hubsync-ai-assistant',
    '_id': '1%3A0%3A4vzEJYsBwWA-qbOZXNiF',
    '_score': 0.18232156,
    '_source': {'vector_field': [0.0003570442275472481,
      -0.012835217780512193,
      4.022619784846402e-06,
      -0.021260625304497478,
      0.009574631570594495,
      0.014164841792679997,
      -0.00951449793830079,
      -0.011231650837830825,
      0.006621395295545323,
      -0.03533192506354503,
      0.02982634681025395,
      0.02418713495160791,
      0.00029336088858287877,
      0.016343020758665967,
      -0.004279519633347607,
      -0.013369740094739313,
      0.022890918409456326,
      0.021674881285459708,
      -0.01737197567804273,
      -0.0034643728480375532,
      -0.026913199778621003,
      -0.0006978021854047413,
      -0.022730561435791437,
      0.0142583828